# Método del gradiente

In [ ]:
import sympy as sp
import numpy as np
import pandas as pd

# Búsqueda inexacta en línea para el paso:
def backtrackingline(func, gradient, xk, direction, lamb = 0.5, c1 = 10e-4, max_iter = 100): 
    
    def armijo_condition(lamb):
        x_new = xk + lamb * direction
        func_value = func(*x_new) 
        return (func_value <= func(*xk) + c1 * lamb * gradient.dot(direction))
    
    for _ in range(max_iter):
        if armijo_condition(lamb):
            break
        lamb = lamb / 2  # Reducción del paso si no se cumple la condición de Armijo

    return lamb 

# Test de convergencia de descenso lineal
def test_convergence_linear_descend(x0, x_new, func, gradient_point_new, tol1, tol2, tol3):

    conditions = (np.linalg.norm(x0 - x_new) < tol1,
                  np.linalg.norm(gradient_point_new) < tol2,
                  abs(func(*x_new) - func(*x0)) < tol3)
    
    test = any(conditions)
    condition_idx = np.where(conditions)[0]

    return test, condition_idx

# Método del gradiente
def gradient_linear_descend(f, x, x0, tol1 = 1e-4, tol2 = 1e-4, tol3 = 1e-4, max_iterations = 100):
    
    
    func = sp.lambdify(x, f, 'numpy')
    gradient = [sp.diff(f, xi) for xi in x] # expresión simbólica del gradiente
    gradient_func = sp.lambdify(x, gradient, 'numpy')
    
    gradient_point = gradient_func(*x0)
    gradient_point = np.array(gradient_point) # pasamos a numpy array para poder operar sobre él
    d0 = -gradient_point # dirección inicial
    
    iteration = 0
    results = []
    
    results.append([iteration, np.round(x0, 5)])
    
    while iteration < max_iterations:

        lamb = backtrackingline(func, gradient_point, x0, d0) # obtención del paso con Backtracking LineSearch
        
        x_new = x0 + lamb * d0 # Aquí se obtiene el x_{k + 1}
        gradient_point_new = np.array(gradient_func(*x_new))
        d_new = -gradient_point_new
        
        test, cond_idx = test_convergence_linear_descend(x0, x_new, func, gradient_point_new, tol1, tol2, tol3)
        
        if test:
            print(f"Convergencia conseguida debido a la condición: {cond_idx}")
            break
        else:
            x0 = x_new
            d0 = d_new
            gradient_point = gradient_point_new
            iteration = iteration + 1

        results.append([iteration, np.round(x0, 5)])
        
    table = pd.DataFrame(results)
    table.columns = ['Iteration', 'x']
    
    return table, x_new
    
# Llamada a función

x, y = sp.symbols('x y')
f = 9*x**2 + 2*x*y + y**2

initial_guess = [0, -1]

result, xopt = gradient_linear_descend(f, (x, y), initial_guess)
print(result)

print(f"\nEl valor óptimo corresponde con el x^(k + 1) de la última iteración: x* ≈ {xopt}")

# Método de Newton (multivariable)

In [ ]:
import sympy as sp
import numpy as np

def newtons_method(f, x, initial_guess, tol = 1e-6, max_iter = 100):
    
    """"
    Método de Newton para hallar mínimos
    
    Parámetros:
    f: function to minimize
    x: Lista de símbolos Sympy
    initial_guess: Punto inicial
    tol: tolerancia. La condición se da cuando la norma del gradiente sea menor que cierto nivel
    max_iter: Máximo número de iteraciones
    """
    
    # Cálculo simbólico de gradiente y hessiano
    gradient = [sp.diff(f, xi) for xi in x]
    hessian = sp.hessian(f, x)
    
    # Paso a numpy para posterior sustitución de valores
    
    gradient_func = sp.lambdify(x, gradient, 'numpy')
    hessian_func = sp.lambdify(x, hessian, 'numpy')
    
    x0 = initial_guess
    results = []
    
    for iteration in range(max_iter):
        grad = gradient_func(*x0)
        hess = hessian_func(*x0)
        step = -np.dot(np.linalg.inv(hess), grad)
        
        results.append([iteration, np.round(x0, 4)])
        
        if np.linalg.norm(step) < tol:
            break
        
        x0 = x0 + step
        
    table = pd.DataFrame(results)
    table.columns = ['Iteration', 'x']
    
    return table


# Llamada a función

x, y = sp.symbols('x y')
f = 3*x**2 - 2*x*y + y**2 + x

initial_guess = [1, 1]

result = newtons_method(f, (x, y), initial_guess)
result

# Método de la razón aúrea

In [ ]:
import math
import numpy as np

def golden_ratio_search(func, lower, upper, tolerance, max_iter = 100):

    golden_ratio = (math.sqrt(5) - 1) / 2

    # Cálculo de subintervalos iniciales
    a = lower
    b = upper
    x1 = a + (1 - golden_ratio) * (b - a)
    x2 = a + golden_ratio * (b - a)

    results = []
    iter = 0
    
    results.append([iter, np.round(a, 4), np.round(b, 4)])
    
    while abs(b - a) > tolerance and (iter < max_iter):
        if func(x1) < func(x2):
            b = x2 # a no se ve modificado
        else:
            a = x1 # b no se ve modificado

        x1 = a + (1 - golden_ratio) * (b - a)
        x2 = a + golden_ratio * (b - a)
        iter = iter + 1
        
        results.append([iter, np.round(a, 4), np.round(b, 4)])
    
    table = pd.DataFrame(results)
    table.columns = ['Iteration', 'a', 'b']
        
    # Devolución del punto mínimo aproximado y del valor de la función ahí
    min_point = (a + b) / 2
    min_value = func(min_point)

    return table, min_point, min_value, iter


def test_function(x):
    return -x * math.cos(x)

lower_bound = 0
upper_bound = math.pi/2
tolerance = 1e-4

results, min_point, min_value, iter = golden_ratio_search(test_function, lower_bound, upper_bound, tolerance)
print(f"Punto en el mínimo: {min_point}")
print(f"Valor de la función en el mínimo: {min_value}")

results

# Método de Newton (1D)

In [ ]:
import sympy as sp

def newton_optimization_1d(f, x0, tol = 1e-4, max_iter = 100):
    
    x = sp.symbols('x')
    f_expr = sp.sympify(f)
    
    # Cálculo de primera y segunda derivada
    
    f_prime = f_expr.diff(x)
    f_double_prime = f_prime.diff(x)
    
    results = []
    x_current = x0
    iteration = 0
    
    results.append([iteration, x_current])
    
    while iteration < max_iter:
        df_prime = f_prime.subs(x, x_current)
        df_double_prime = f_double_prime.subs(x, x_current)
        
        x_new = x_current - df_prime / df_double_prime # iteración
        
        if abs(x_new - x_current) < tol: # salida por proximidad en iteraciones sucesivas para el punto
            
            # Criterio de segunda derivada (cuidado con algunas funciones)
            
            second_derivative = df_double_prime.subs(x, x_new)
            
            if second_derivative > 0:
                print(f"Se ha encontrado un mínimo.")
            elif second_derivative < 0:
                print(f"Se ha encontrado un máximo.")
            else:
                print(f"Según el criterio de la segunda derivada, no es máximo ni mínimo.")
            
            break
        
        x_current = x_new
        iteration += 1
        results.append([iteration, x_current])
        
    table = pd.DataFrame(results)
    table.columns = ['Iteration', 'x']
        
    return table

# main 

x = sp.symbols('x')
f = x ** 4 - 14*x**3 + 60*x**2 - 70*x
x0 = 0.7 # valor inicial
    
results = newton_optimization_1d(f, x0, tol = 0.001, max_iter = 100)
results

# Método de la bisección

In [ ]:
import sympy as sp

def bisect_minimum(func, a, b, tol = 1e-6, max_iter = 100):
    """
    Hallar el mínimo de una función de una sola variable en un intervalo [a, b] haciendo uso de su derivada.

    Parámetros
    func: función simbólica
    a, b: extremo izquierdo y derecho del intervalo
    tol: tolerancia
    max_iter: número máximo de iteraciones
    """

    x = sp.symbols('x')
    derivative = sp.diff(func, x)
    
    derivative = sp.lambdify(x, derivative, 'numpy')

    if derivative(a) * derivative(b) >= 0:
        raise ValueError("Para aplicar el método de la bisección es necesario que las derivadas en los extremos tengan signo contrario.")

    iteration = 0
    c = (a + b) / 2
    results = []
    results.append([iteration, c])
    
    while (b - a) / 2 > tol and iteration < max_iter:
        
        if derivative(c) == 0:
            break
        elif derivative(c) * derivative(a) < 0:
            b = c
        else:
            a = c
        
        c = (a + b) / 2
        iteration += 1
        results.append([iteration, c])
     
    table = pd.DataFrame(results)
    table.columns = ['Iteration', 'x']
    
    return table

# Example usage:
x = sp.symbols('x')
func = x**4 - 14*x**3 + 60*x**2 - 70*x  # A simple quadratic function with a minimum at x = -1

results = bisect_minimum(func, 0, 2)
results


# Método del gradiente conjugado

In [ ]:
import numpy as np

def conjugate_gradient(A, b, x0, tol = 1e-6, max_iter = 100):
    
    x = x0
    r = np.dot(A, x) - b 
    d = -r
    
    results = []
    
    for iteration in range(max_iter):
        
        results.append([iteration, np.round(x, 4)])
        
        Ad = np.dot(A, d)
        lamb = np.dot(r, r) / np.dot(d, Ad)
        x = x + lamb * d
        r_new = np.dot(A, x) - b
        
        if np.linalg.norm(r_new) < tol:
            results.append([iteration + 1, np.round(x, 4)]) # x_{k + 1} retornado
            break
        
        beta = np.dot(r_new, r_new) / np.dot(r, r)
        d = -r_new + beta * d
        r = r_new
        
    table = pd.DataFrame(results)
    table.columns = ['Iteration', 'x']
    return table

# main
A = np.array([[4, -1, 1], [-1, 4, -2], [1, -2, 4]]) # entre corchetes por filas
b = np.array([12, -1, 5])
x0 = np.array([0, 0, 0])

results = conjugate_gradient(A, b, x0)
results


# Método del gradiente conjugado (no lineal) - Fletcher-Reeves

In [ ]:
import sympy as sp
import numpy as np
import pandas as pd

def backtrackingline(func, gradient, xk, direction, lamb = 0.5, c1 = 10e-4, max_iter = 100): 
    
    def armijo_condition(lamb):
        x_new = xk + lamb * direction
        func_value = func(*x_new) 
        return (func_value <= func(*xk) + c1 * lamb * gradient.dot(direction))
    
    for _ in range(max_iter):
        if armijo_condition(lamb):
            break
        lamb = lamb / 2  # Reducción del paso si no se cumple la condición de Armijo

    return lamb 

def fletcher_reeves(f, x, x0, tol = 1e-3, max_iterations = 100):
    results = []
    
    gradient = [sp.diff(f, xi) for xi in x]
    func = sp.lambdify(x, f, 'numpy')
    gradient_func = sp.lambdify(x, gradient, 'numpy')
    
    gradient_point = gradient_func(*x0)
    gradient_point = np.array(gradient_point)
    d0 = -gradient_point
    
    # Inicialización de variables para el bucle
    iteration = 0
    x_new = x0
    d_new = d0
    gradient_point_new = gradient_point
    results.append([iteration, np.round(x_new, 5)])
    
    while (np.linalg.norm(gradient_point) > tol and iteration < max_iterations):
        
        lamb = backtrackingline(func, gradient_point_new, x_new, d_new)
        
        gradient_point_old = gradient_func(*x_new) # Este es el gradiente evaluado en el x_k
        
        x_new = x_new + lamb*d_new # Aquí se pone el x_{k + 1}
        
        gradient_point_new = gradient_func(*x_new)
        gradient_point_new = np.array(gradient_point_new)
        
        beta_new = np.dot(gradient_point_new, gradient_point_new) / np.dot(gradient_point_old, gradient_point_old)
        d_new = -gradient_point_new + beta_new * d_new
        
        iteration = iteration + 1
        results.append([iteration, np.round(x_new, 5)])
        
    table = pd.DataFrame(results)
    table.columns = ['Iteration', 'x']
    return table
    
# Llamada a función

x, y = sp.symbols('x y')
f = x - y + 2*x**2 + 2*x*y + y**2

initial_guess = [0, 0]

result = fletcher_reeves(f, (x, y), initial_guess)
result

In [ ]:
from numpy.linalg import norm
import numpy as np
import sympy as sp

def levenberg_marquardt(func, indep_vars_symbols, params_symbols, indep_vars_values, y_vec, init_vals, tol = 1e-4, max_iter = 10, lambda_0 = 10):
    
    # Obtenemos la lista de funciones evaluados en todos los puntos
    # de variables independientes
    f = []
    for idx in range(len(indep_vars_values[indep_vars_symbols[0]])):
        subs_values = {key: values[idx] for key, values in indep_vars_values.items()}
        f.append(func.subs(subs_values))
    # Residuos
    r = [f_j - y_j for f_j, y_j in zip(f, y_vec)]
    # Jacobiano
    jacobo = sp.Matrix(f).jacobian(params_symbols)

    # Inicializamos variables para el bucle
    results = []
    params_k = init_vals
    lambda_k = lambda_0
    diff_cond = 2 * tol
    iter_count = 0
    last_iter = False

    while iter_count < max_iter:

        # Valores de la iteración k
        jacobo_k = jacobo.subs(params_k)
        r_k = np.asarray([r_j.subs(params_k) for r_j in r], dtype = np.float16)
        d_k = - np.dot(
            sp.Matrix(
                np.dot(jacobo_k.T, jacobo_k) +
                lambda_k * np.diag(np.dot(jacobo_k.T, jacobo_k).diagonal())).inv(),
            np.dot(jacobo_k.T, r_k))
        
        # Valores de la iteración k + 1
        params_k1 = {key_j: value_j + d_k_j for key_j, value_j, d_k_j in zip(params_k.keys(), params_k.values(), d_k)}
        r_k1 = np.asarray([r_j.subs(params_k1) for r_j in r], dtype = np.float16)

        # Añadimos la iteración a la lista de resultados
        results.append([iter_count] + list(params_k.values()) + [sum(r_k**2), lambda_k] + list(d_k))

        # Condición para el siguiente paso
        lambda_cond = norm(r_k1) ** 2 - norm(r_k)**2
        if lambda_cond > 0:
            lambda_k *= 10
        else:
            lambda_k /= 10
            params_k = params_k1
            diff_cond = lambda_cond
        iter_count += 1

        # Condición de salida
        if last_iter:
            break
        if abs(diff_cond) < tol:
            last_iter = True

    # Generamos tabla con los resultados
    table = pd.DataFrame(results)
    table.columns = ['Iter'] + [str(symbol) for symbol in params_symbols] + ['\u2211 r²', '\u03BB'] + ['d_' + str(symbol) for symbol in params_symbols]
    
    return table, params_k



# Ejemplo diapositivas

# Datos
# Variables independientes
t = sp.symbols('t')
indep_vars_symbols = [t]
indep_vars_values = {t: [0, 0.5, 1, 1.5, 2, 2.5, 3]}
y_vec = [1.145, 0.512, 0.401, 0.054, 0.038, 0.014, 0.046]

# Ecuación de ajuste
# Parámetros
x = sp.symbols('x')
params_symbols = [x]
func = np.e ** (- x * t)

# Valores iniciales para los parámetros
init_vals = {x: 4}

# Llamamos a la función
results_table, x_sol = levenberg_marquardt(func, indep_vars_symbols, params_symbols, indep_vars_values, y_vec, init_vals)
results_table


# Práctica 1: Ejercicio 2

# Datos
# Variables independientes
t = sp.symbols('t')
indep_vars_symbols = [t]
indep_vars_values = {t: [1, 2, 3, 4, 5, 6, 7, 8]}
y_vec = [8.3, 11.0, 14.7, 19.7, 26.7, 35.2, 44.4, 55.9]

# Ecuación de ajuste
# Parámetros
x1, x2 = sp.symbols('x1 x2')
params_symbols = [x1, x2]
func = x1 * np.e ** (x2 * t)

# Valores iniciales para los parámetros
init_vals = {x1: 4, x2: 0.3}

# Llamamos a la función
results_table, x_sol = levenberg_marquardt(func, indep_vars_symbols, params_symbols, indep_vars_values, y_vec, init_vals)
results_table

# Ejercicio en el tema (2D)

# Datos
# Variables independientes
t = sp.symbols('t')
indep_vars_symbols = [t]

indep_vars_values = {t: list(np.arange(-5, 5, 0.5))}
y_vec = [-0.99999, -1.2090, -1.3877, -1.5090, -1.5510, -1.5, -1.3510, -1.109, -0.7877, -0.409, 0, 0.4090, 0.787786, 1.10901,
         1.3510, 1.5, 1.5510, 1.5090, 1.3877, 1.209, 0.9999]

# Ecuación de ajuste
# Parámetros
x1, x2 = sp.symbols('x1 x2')
params_symbols = [x1, x2]
func = sp.sin(x1 * t) + x2 * t

# Valores iniciales para los parámetros
init_vals = {x1: 0.5, x2: 0.3}

# Llamamos a la función
results_table, x_sol = levenberg_marquardt(func, indep_vars_symbols, params_symbols, indep_vars_values, y_vec, init_vals)
results_table